# WEST IC C5 Conditioning Campaign 

In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
import matplotlib.dates as mdates
import datetime

In [3]:
import seaborn as sns

# Creating the database

In [4]:
def read_conditoning_data(filename):
    """
    Import and return the ICRH Conditioning data into a pandas DataFrame
    """

    data = pd.read_csv(filename, delimiter='\t', skiprows=18, 
                names=('Temps',
                       'PiG','PrG','PiD','PrD',
                       'V1','V2','V3','V4', 
                       'Ph(V1-V3)','Ph(V2-V4)',
                       'Consigne_mes', 'Vide_gauche', 'Vide_droit',
                       'reserve1', 'reserve2', '_'),
                index_col='Temps') # last element '_' to avoid pandas crashing
    # convert phase in degree and wrap it between 0° and 359°
    data['Ph(V1-V3)'] /= 100
    data['Ph(V2-V4)'] /= 100
    #data['Ph(V1-V3)'] %= 360
    #data['Ph(V2-V4)'] %= 360
    return data 

def read_conditioning_metadata(filename):
    """
    Import and return the ICRH Conditioning metadata into a dictionary
    """
    para_dic = {}
    with  open(filename,'r') as cmt_file:    # open file
        for line in cmt_file:    # read each line
            if line[0] == '#':    # check the first character
                line = line[1:]    # remove first '#'
                para = line.split('=')     # seperate string by '='
                if len(para) == 2:
                    para_dic[ para[0].strip()] = para[1].strip()
    return para_dic 

In [5]:
file = 'Conditionning/2020-12-01_09-37-26.csv'
'''
Get the metadata of a file and generate a pandas DataFrame of them
'''
def forge_df(file):
    metadata = read_conditioning_metadata(file)
    _datetime = pd.datetime.strptime(metadata['Date']+' '+metadata['Heure'], '%Y-%m-%d %H-%M-%S')
    _row = pd.DataFrame(data={'Date':_datetime, 
                       'Antenna':int(metadata['Antenne'][1]), 
                       'Duration':float(metadata['Duree du tir'].split()[0]),
                       'Frequency':float(metadata['Frequence'].split()[0]),
                       'Nb of Vr/Vi':int(metadata['Nb interruptions TOS']),
                       'Nb of SHAD': int(metadata['Nb interruptions SHAD']),
                       'Nb of Optical':int(metadata['Nb interruptions OPT']),
                       'Nb total': int(metadata['Nb interruptions TOTAL']),
                       'Vacuum stop': int(metadata['Arret sur defaut vide'])
                      }, index=[0])#.set_index('Date')

    # pass for empty pulses 
    try:
        _all_data = read_conditoning_data(file)
        # calculation real pulse duration
        t_start = _all_data.query('V1 > 1500 or V2 > 1500 or V3 > 1500 or V4 > 1500').index[0]
        t_stop = _all_data.query('V1 > 1500 or V2 > 1500 or V3 > 1500 or V4 > 1500').index[-1]
        duration = (t_stop - t_start)/1e6  # s
        
        # keep only max values of each columns
        _data_max = _all_data.max().to_frame().T
        # power must be divided by 10 to get kW and divided by 2 as it correspond to total power 
        _data_max['PiG'] = _data_max['PiG']/10/2
        _data_max['PiD'] = _data_max['PiD']/10/2     
        # Max Energy
        _data_max['Energie_Left'] = np.trapz(_data_max.PiG*1e3, _data_max.index/1e6)
        _data_max['Energie_Right'] = np.trapz(_data_max.PiD*1e3, _data_max.index/1e6)
                
        _data_max.columns = [colname+'_max' for colname in _data_max.columns]
        
        # mean values during RF pulse
        _data_mean = _all_data.query('index >= @t_start and index <= @t_stop').mean().to_frame().T
        # power must be divided by 10 to get kW and divided by 2 as it correspond to total power 
        _data_mean['PiG'] = _data_mean['PiG']/10/2
        _data_mean['PiD'] = _data_mean['PiD']/10/2     
        # Mean Energy
        _data_mean['Energie_Left'] = np.trapz(_data_mean.PiG*1e3, _data_mean.index/1e6)
        _data_mean['Energie_Right'] = np.trapz(_data_mean.PiD*1e3, _data_mean.index/1e6)
        
        _data_mean.columns = [colname+'_mean' for colname in _data_mean.columns]
               
        # final data
        _row_final = pd.concat([_data_max, _data_mean, _row], axis=1)
        # Outgassing 
        _row_final['outgassing_gauche'] = np.trapz(_all_data['Vide_gauche'], _all_data.index/1e6)
        _row_final['outgassing_droite'] = np.trapz(_all_data['Vide_droit'], _all_data.index/1e6)

        _row_final['Duration_'] = duration                                            
        _row_final.set_index('Date')
        
    except IndexError:
        _row_final = None
   
    return _row_final

In [6]:
files = glob('../WEST_C4/Conditionning/2019*.csv')

df = pd.DataFrame()
for file in tqdm(files):
    df=df.append(forge_df(file))
df2019=df.set_index('Date')


  0%|                                                                                         | 0/1148 [00:00<?, ?it/s]C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  import sys
100%|██████████████████████████████████████████████████████████████████████████████| 1148/1148 [00:41<00:00, 27.68it/s]


In [7]:
files = glob('Conditionning/2020*.csv')

df = pd.DataFrame()
for file in tqdm(files):
    df=df.append(forge_df(file))
df2020=df.set_index('Date')


  0%|                                                                                          | 0/924 [00:00<?, ?it/s]C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  import sys
100%|████████████████████████████████████████████████████████████████████████████████| 924/924 [00:32<00:00, 28.60it/s]


In [8]:
# merge 2019 and 2020 data
df = pd.concat([df2019, df2020])

In [9]:
df.columns

Index(['PiG_max', 'PrG_max', 'PiD_max', 'PrD_max', 'V1_max', 'V2_max',
       'V3_max', 'V4_max', 'Ph(V1-V3)_max', 'Ph(V2-V4)_max',
       'Consigne_mes_max', 'Vide_gauche_max', 'Vide_droit_max', 'reserve1_max',
       'reserve2_max', '__max', 'Energie_Left_max', 'Energie_Right_max',
       'PiG_mean', 'PrG_mean', 'PiD_mean', 'PrD_mean', 'V1_mean', 'V2_mean',
       'V3_mean', 'V4_mean', 'Ph(V1-V3)_mean', 'Ph(V2-V4)_mean',
       'Consigne_mes_mean', 'Vide_gauche_mean', 'Vide_droit_mean',
       'reserve1_mean', 'reserve2_mean', '__mean', 'Energie_Left_mean',
       'Energie_Right_mean', 'Antenna', 'Duration', 'Frequency', 'Nb of Vr/Vi',
       'Nb of SHAD', 'Nb of Optical', 'Nb total', 'Vacuum stop',
       'outgassing_gauche', 'outgassing_droite', 'Duration_'],
      dtype='object')

In [67]:
df['V_max'] = df[['V1_max','V2_max', 'V3_max', 'V4_max']].max(axis=1)/1e3
df['P_max'] = df['PiD_max'] + df['PiG_max']
df['V_mean'] = df[['V1_mean','V2_mean', 'V3_mean', 'V4_mean']].max(axis=1)/1e3
df['P_mean'] = df['PiD_mean'] + df['PiG_mean']

df['pressure_gauche_mean'] = np.power(10, 1.667*df.Vide_gauche_mean*1e-3 - 9.33)
df['pressure_droit_mean'] = np.power(10, 1.667*df.Vide_droit_mean*1e-3 - 9.33)
df['pressure_mean'] = df[['pressure_gauche_mean', 'pressure_droit_mean']].max(axis=1)

df['pressure_gauche_max'] = np.power(10, 1.667*df.Vide_gauche_max*1e-3 - 9.33)
df['pressure_droit_max'] = np.power(10, 1.667*df.Vide_droit_max*1e-3 - 9.33)
df['pressure_max'] = df[['pressure_gauche_max', 'pressure_droit_max']].max(axis=1)

df['year'] = pd.to_datetime(df.index).year
df['day'] = pd.to_datetime(df.index).day

df['SWR'] = df['Nb of Vr/Vi'] > 0
df['optical'] = df['Nb of Optical'] > 0
df['SHAD'] = df['Nb of SHAD'] > 0
df['pressure'] = df['Vacuum stop'] > 0
df['date'] = pd.to_datetime(df.index)
# max outgassing (left vs right)
df['outgassing'] = df[['outgassing_droite', 'outgassing_gauche']].max(axis=1)
# outgassing "rate"
df['outgassing_rate'] = df['outgassing']/df['Duration_']


In [68]:
#df.to_csv('C4_C5_Vacuum_Conditionning_Data.csv')

# Ploting the results

# Duration vs date

### Q4 - 2019

#### outgassing vs duration

In [12]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['outgassing'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['outgassing'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['outgassing'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['outgassing'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['outgassing'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_yscale('log')
# ax.set_ylim(top=22)  # half-side only
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Outgassing [Pa.s]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02, 0.05, 0.1, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### outgassing vs voltage

In [69]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019')
ax.scatter(_df['V_mean'][_df['Nb of SHAD'] > 0], _df['outgassing_rate'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['V_mean'][_df['Nb of Optical'] > 0], _df['outgassing_rate'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['V_mean'][_df['Nb of Vr/Vi'] > 0], _df['outgassing_rate'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['V_mean'][_df['Vacuum stop'] > 0], _df['outgassing_rate'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['V_mean'], _df_nostop['outgassing_rate'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

#ax.set_xscale('log')
#ax.set_xlim(left=1e-3, right=6)
ax.set_yscale('log')
# ax.set_ylim(top=22)  # half-side only
#ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Mean Voltage [kV]', fontsize=18)
ax.set_ylabel('Outgassing [Pa.s]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02, 0.05, 0.1, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### P mean vs duration

In [13]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['P_mean'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['P_mean'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['P_mean'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['P_mean'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['P_mean'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
# ax.set_yscale('log')
ax.set_ylim(top=22)  # half-side only
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Total Fwd. Power [kW]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02, 0.05, 0.1, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### P max vs duration

In [14]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['P_max'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['P_max'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['P_max'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['P_max'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['P_max'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 20)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Max Power [kW]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02, 0.5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### V max vs duration

In [15]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['V_max'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['V_max'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['V_max'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['V_max'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['V_max'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 29)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Max Voltage[kV]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02, 0.5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Q1 - 2020

#### outgassing vs duration

In [16]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [1] and year == 2020')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['outgassing'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['outgassing'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['outgassing'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['outgassing'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['outgassing'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_yscale('log')
# ax.set_ylim(top=22)  # half-side only
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Outgassing [Pa.s]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q1 - 2020')
for target_duration in [0.02, 0.05, 0.1, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### V mean vs duration

In [17]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [1] and year == 2020')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['V_mean'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['V_mean'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['V_mean'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['V_mean'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['V_mean'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 29)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Mean Voltage[kV]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02, 0.5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### V max vs duration

In [18]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [1] and year == 2020')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['V_max'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['V_max'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['V_max'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['V_max'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['V_max'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 29)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Max Voltage[kV]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q1 - 2020')
for target_duration in [0.02, 0.5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### P max vs duration

In [19]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [1] and year == 2020')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['P_max'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['P_max'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['P_max'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['P_max'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['P_max'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 21)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Max Power [kW]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q1 - 2020')
for target_duration in [0.02,  0.5, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### P mean vs duration

In [20]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [1] and year == 2020')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['P_mean'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['P_mean'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['P_mean'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['P_mean'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['P_mean'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 21)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Mean Power [kW]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02,  0.5, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Pressure mean vs duration

In [21]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [1] and year == 2020')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['pressure_mean'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['pressure_mean'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['pressure_mean'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['pressure_mean'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['pressure_mean'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_yscale('log')
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Mean Pressure [Pa]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02,  0.5, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Pressure max vs duration

In [22]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019')
ax.scatter(_df['Duration_'][_df['Nb of SHAD'] > 0], _df['pressure_max'][df['Nb of SHAD'] > 0], alpha=.8, marker='o', facecolor='none', color='k', s=70, label='SHAD Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Optical'] > 0], _df['pressure_max'][df['Nb of Optical'] > 0], marker='d', color='C1', s=90, label='Optical Arc Detection')
ax.scatter(_df['Duration_'][_df['Nb of Vr/Vi'] > 0], _df['pressure_max'][df['Nb of Vr/Vi'] > 0], marker='v', color='C3', alpha=.8, s=60, label='Vr/Vi Arc Detection')
ax.scatter(_df['Duration_'][_df['Vacuum stop'] > 0], _df['pressure_max'][df['Vacuum stop'] > 0], marker='s', color='C4', alpha=.8, s=60, label='Vacuum stop')

_df_nostop = _df.query("`Vacuum stop` == 0 and `Nb total` == 0 ")
ax.scatter(_df_nostop['Duration_'], _df_nostop['pressure_max'],  marker='o', s=60, color='green', alpha=0.5, label='No Interrupt')

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_yscale('log')
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Max Pressure [Pa]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
# ax.set_title('Q4 - 2019')
for target_duration in [0.02,  0.5, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Chronologie

#### Q1 2020

In [94]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [1] and year == 2020')
_min = _df['date'].min().value
_max = _df['date'].max().value
_norm = plt.Normalize(vmin=_min, vmax=_max)
_cmap = 'plasma'
sm = plt.cm.ScalarMappable(cmap=_cmap, norm=_norm)

_df_ = _df.query(' SWR == True')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='v', edgecolor='r', s=120, norm=_norm, label='$P_r/P_i$', cmap=_cmap)

_df_ = _df.query('optical == True')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='d', edgecolor='k', s=70, norm=_norm, label='Optical', cmap=_cmap)

_df_ = _df.query('SHAD == True')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='X', edgecolor='k', s=70, norm=_norm, label='SHAD', cmap=_cmap)

_df_ = _df.query('pressure == True')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='s', edgecolor='k', s=70, norm=_norm, label='Pressure', cmap=_cmap)

_df_ = _df.query(' SWR == False and optical == False and SHAD == False and pressure == False')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='o', edgecolor='g', s=70, norm=_norm, label='No interlock', cmap=_cmap)

cbar = plt.colorbar(sm)
# Change the numeric ticks into ones that match the x-axis
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%H:%M'))
cbar.set_label('Time (HH:MM)', fontsize=14)

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 28)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Mean Voltage [kV]', fontsize=18)
ax.legend(fontsize=14)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q1 Antenna Vacuum Conditionning')
#for target_duration in [0.02,  0.1, 0.5, 1, 2, 5]:
#    ax.axvline(target_duration, color='gray', ls='--', label='target')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator


In [88]:
_df['pressure_mean'].min()

1.2494489027721476e-05

#### Outgassing rate vs voltage

In [91]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [1] and year == 2020')
_min = _df['date'].min().value
_max = _df['date'].max().value
_norm = plt.Normalize(vmin=_min, vmax=_max)
_cmap = 'plasma'
sm = plt.cm.ScalarMappable(cmap=_cmap, norm=_norm)

_df_ = _df.query(' SWR == True')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='v', edgecolor='r', s=120, norm=_norm, label='SWR', cmap=_cmap)

_df_ = _df.query('optical == True')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='d', edgecolor='k', s=70, norm=_norm, label='Optical', cmap=_cmap)

_df_ = _df.query('SHAD == True')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='X', edgecolor='k', s=70, norm=_norm, label='SHAD', cmap=_cmap)

_df_ = _df.query('pressure == True')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='s', edgecolor='k', s=70, norm=_norm, label='Pressure', cmap=_cmap)

_df_ = _df.query(' SWR == False and optical == False and SHAD == False and pressure == False')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='o', edgecolor='g', s=70, norm=_norm, label='No interlock', cmap=_cmap)

cbar = plt.colorbar(sm)
# Change the numeric ticks into ones that match the x-axis
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%H:%M'))

ax.set_yscale('log')
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_ylabel('Outgassing ratio [Pa.s/s]', fontsize=18)
ax.set_xlabel('Mean Voltage [kV]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q1 Antenna Vacuum Conditionning')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator


Text(0.5, 1.0, 'Q1 Antenna Vacuum Conditionning')

#### Q4 2019

In [55]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019')
_min = _df['date'].min().value
_max = _df['date'].max().value
_norm = plt.Normalize(vmin=_min, vmax=_max)
_cmap = 'plasma'
sm = plt.cm.ScalarMappable(cmap=_cmap, norm=_norm)

_df_ = _df.query(' SWR == True')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='v', edgecolor='r', s=120, norm=_norm, label='SWR', cmap=_cmap)

_df_ = _df.query('optical == True')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='d', edgecolor='k', s=70, norm=_norm, label='Optical', cmap=_cmap)

_df_ = _df.query('SHAD == True')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='X', edgecolor='k', s=70, norm=_norm, label='SHAD', cmap=_cmap)

_df_ = _df.query('pressure == True')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='s', edgecolor='k', s=70, norm=_norm, label='Pressure', cmap=_cmap)

_df_ = _df.query(' SWR == False and optical == False and SHAD == False and pressure == False')
ax.scatter(x=_df_['Duration_'], y=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='o', edgecolor='g', s=70, norm=_norm, label='No interlock', cmap=_cmap)

cbar = plt.colorbar(sm)
# Change the numeric ticks into ones that match the x-axis
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%H:%M'))
cbar.set_label('Time (hours:minutes)', fontsize=14)

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 28)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
ax.set_ylabel('Mean Voltage [kV]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 Antenna Vacuum Conditionning')
for target_duration in [0.02,  0.1, 0.5, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator


#### outgassing vs voltage

In [78]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019 and day > 10')
_min = _df['date'].min().value
_max = _df['date'].max().value
_norm = plt.Normalize(vmin=_min, vmax=_max)
_cmap = 'plasma'
sm = plt.cm.ScalarMappable(cmap=_cmap, norm=_norm)

_df_ = _df.query(' SWR == True')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='v', edgecolor='r', s=120, norm=_norm, label='SWR', cmap=_cmap)

_df_ = _df.query('optical == True')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='d', edgecolor='k', s=70, norm=_norm, label='Optical', cmap=_cmap)

_df_ = _df.query('SHAD == True')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='X', edgecolor='k', s=70, norm=_norm, label='SHAD', cmap=_cmap)

_df_ = _df.query('pressure == True')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='s', edgecolor='k', s=70, norm=_norm, label='Pressure', cmap=_cmap)

_df_ = _df.query(' SWR == False and optical == False and SHAD == False and pressure == False')
ax.scatter(y=_df_['outgassing_rate'], x=_df_['V_mean'], c=_df_['date'],
    alpha=.8, marker='o', edgecolor='g', s=70, norm=_norm, label='No interlock', cmap=_cmap)

cbar = plt.colorbar(sm)
# Change the numeric ticks into ones that match the x-axis
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%H:%M'))

ax.set_yscale('log')
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_ylabel('Outgassing ratio [Pa.s/s]', fontsize=18)
ax.set_xlabel('Mean Voltage [kV]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 Antenna Vacuum Conditionning')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator


Text(0.5, 1.0, 'Q4 Antenna Vacuum Conditionning')

In [25]:
fix, ax = plt.subplots(figsize=(8,6))
_df=df.query('Duration_>0 and Antenna == [4] and year == 2019 and day > 10')
_df.plot(kind='scatter', ax=ax, x='Duration_', y='V_mean', alpha=.8, marker='o', color=_df.reset_index()['Date'], s=70,
        cmap='tab20', colorbar=True)

ax.set_xscale('log')
ax.set_xlim(left=1e-3, right=6)
ax.set_ylim(0, 28)
ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
ax.set_xlabel('Pulse Duration [s]', fontsize=18)
# ax.set_ylabel('Mean Power [kW]', fontsize=18)
ax.legend(fontsize=12)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.set_title('Q4 - 2019')
for target_duration in [0.02,  0.5, 1, 2, 5]:
    ax.axvline(target_duration, color='gray', ls='--', label='target')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\plotting\_matplotlib\core.py:227: UserWarning: 'color' and 'colormap' cannot be used simultaneously. Using 'color'
  "'color' and 'colormap' cannot be used simultaneously. Using 'color'"
No handles with labels found to put in legend.


### V max vs duration

In [62]:
fig, ax = plt.subplots(figsize=(8,6))
_df = df.query('Duration_>0 and Antenna == [1] and year == 2020')
ax.set_title('Q1 - 2020')

_min = _df['date'].min().value
_max = _df['date'].max().value
_norm = plt.Normalize(vmin=_min, vmax=_max)
_cmap = 'plasma'
sm = plt.cm.ScalarMappable(cmap=_cmap, norm=_norm)

_y = 'V_mean'

_df.query('`Nb of Vr/Vi`>0').plot(kind='scatter', ax=ax, x='outgassing', y=_y, 
                                 alpha=.8, marker='v', edgecolor='r', s=120,  label='SWR',  c='date', norm=_norm)
_df.query('`Nb of Optical`>0').plot(kind='scatter', ax=ax, x='outgassing', y=_y, 
                                   alpha=.8, marker='d', edgecolor='k',  label='Optical', s=120,  c='date', norm=_norm)
_df.query('`Nb of SHAD`>0').plot(kind='scatter', ax=ax, x='outgassing', y=_y, 
                               alpha=.8, marker='X', edgecolor='k', label='SHAD', s=70,  c='date', norm=_norm)
_df.query('`Vacuum stop`>0').plot(kind='scatter', ax=ax, x='outgassing', y=_y, 
                                 alpha=.8, marker='s',  label='Pressure stop', s=70,  c='date', norm=_norm)


cbar = plt.colorbar(sm)
# Change the numeric ticks into ones that match the x-axis
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%H:%M'))
cbar.set_label('Time (hours:minutes)', fontsize=14)

ax.set_xscale('log')
# ax.set_xlim(left=1e-3, right=6)
# ax.set_yscale('log')
# ax.set_ylim(top=22)  # half-side only
# ax.set_xlim(right=10)
ax.grid(True)
ax.grid(True, which='minor', alpha=0.4)
# ax.set_xlabel('Pulse Duration [s]', fontsize=18)
# ax.set_ylabel('Fwd. Power [kW]', fontsize=18)
ax.legend(fontsize=16)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator


In [27]:
ax

<AxesSubplot:xlabel='Pulse Duration [s]', ylabel='Fwd. Power [kW]'>

In [28]:
df

,PiG_max,PrG_max,PiD_max,PrD_max,V1_max,V2_max,V3_max,V4_max,Ph(V1-V3)_max,Ph(V2-V4)_max,...,pressure_droit_max,pressure_max,year,day,SWR,optical,SHAD,pressure,date,outgassing
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-05 13:30:41,15.85,124.0,0.55,10.0,1270.0,33162.0,1089.0,1080.0,522.57,350.11,...,0.000039,0.000039,2019,5,False,False,True,False,2019-07-05 13:30:41,58.1905
2019-07-05 13:30:53,18.05,108.0,0.55,10.0,1270.0,35299.0,1089.0,1080.0,520.50,541.30,...,0.000040,0.000040,2019,5,False,False,True,False,2019-07-05 13:30:53,58.1975
2019-07-05 13:31:05,18.45,43.0,0.55,10.0,1270.0,35306.0,1089.0,1080.0,517.12,527.64,...,0.000038,0.000038,2019,5,False,False,True,False,2019-07-05 13:31:05,58.0470
2019-07-05 13:36:08,21.45,50.0,0.55,10.0,1270.0,38108.0,1089.0,1080.0,520.85,272.92,...,0.000037,0.000037,2019,5,False,False,True,False,2019-07-05 13:36:08,57.6535
2019-07-05 13:36:55,21.60,50.0,0.55,10.0,1270.0,38848.0,1089.0,1080.0,520.91,266.78,...,0.000034,0.000034,2019,5,False,False,True,False,2019-07-05 13:36:55,57.6645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-14 16:40:18,7.20,18.0,5.40,31.0,18938.0,21875.0,18375.0,21188.0,176.00,172.00,...,0.003425,0.003425,2020,14,False,False,False,False,2020-12-14 16:40:18,19871.5095
2020-12-14 16:40:44,7.20,26.0,5.40,37.0,17688.0,21438.0,17375.0,20688.0,176.00,248.00,...,0.002849,0.002849,2020,14,False,False,False,False,2020-12-14 16:40:44,19730.5680
2020-12-14 16:50:17,11.60,54.0,8.90,63.0,21375.0,26688.0,21188.0,25438.0,175.00,172.00,...,0.000324,0.000324,2020,14,True,False,False,False,2020-12-14 16:50:17,16713.9285
